# Pequeña práctica con un dataset de noticias en español

El dataset proviene de Kaggle
Se trata de un conjunto de 1000 noticias falsas y 1000 noticias verdaderas.


[Dataset Noticias](https://www.kaggle.com/arseniitretiakov/noticias-falsas-en-espaol?select=fakes1000.csv)

[Basic tasks with spacy](https://necronet.github.io/Spacy-getting-started-in-spanish/)

### Usar amazon reviews y replicar la últimas práctica de Topic Modeling

In [24]:
import pandas as pd
import re
from gensim.corpora import Dictionary
import nltk

In [5]:
df = pd.read_csv('../data/fakes1000.csv')


,class,Text
0,True,Algunas de las voces extremistas más conocida...
1,True,Después de casi dos años y medio de luchas po...
2,True,Dos periodistas birmanos de la agencia Reuter...
3,True,El Cuerpo Nacional de Policía ha detenido a c...
4,True,El desfile de la firma en Roma se convierte e...
...,...,...
1995,True,El Consejo de Gobierno ha dado su visto bueno...
1996,True,Investigadores valencianos han desarrollado u...
1997,True,Los arrestados actuaban en coches y en establ...
1998,True,El Rey ha encargado este miércoles a Pedro Sá...


' Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento de la conocida como alt-right o ligadas a grupos antisemitas o xenófobos , han sido eliminadas de un plumazo de la redes sociales Facebook e Instagram. Se trat'

In [11]:
# For tokenization, tagging, lemmatization, stemming and NER
import spacy
nlp = spacy.load("es_core_news_sm")

In [12]:
sample = df['Text'][0]
print(sample)

 Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento de la conocida como alt-right o ligadas a grupos antisemitas o xenófobos , han sido eliminadas de un plumazo de la redes sociales Facebook e Instagram. Se trat


In [22]:
doc = nlp(sample)
for w in doc:
    print(w.lemma_)

 
alguno
de
el
voz
extremista
más
conocido
de
EE.UU.
,
cuenta
asociado
con
al
movimiento
de
el
conocido
como
alt-right
o
ligado
a
grupo
antisemita
o
xenófobo
,
haber
ser
eliminar
de
uno
plumazo
de
el
red
social
Facebook
e
Instagram
.
él
trat


In [30]:
stopwords = set(nltk.corpus.stopwords.words("spanish"))

False

In [33]:

def normalize_tokenize_lemmatize(text):
    doc = nlp(text)
    # Avoid empty word in beggining with doc[1:]
    return [word.lemma_ for word in doc[1:] if word.lemma_ not in stopwords]
    
    
df['tokens'] = df['Text'].apply(normalize_tokenize_lemmatize)

In [34]:
df['tokens']

0       [alguno, voz, extremista, conocido, EE.UU., ,,...
1       [después, casi, dos, año, medio, lucha, políti...
2       [dos, periodista, birmano, agencia, Reuters, s...
3       [Cuerpo, Nacional, Policía, haber, detener, cu...
4       [desfile, firma, Roma, convertir, oda, liberta...
                              ...                        
1995    [Consejo, Gobierno, haber, dar, visto, bueno, ...
1996    [Investigadores, valenciano, haber, desarrolla...
1997    [arrestado, actuar, coche, establecimiento, Po...
1998    [Rey, haber, encargar, miércoles, Pedro, Sánch...
1999    [prueba, realizado, Centro, Nacional, Microbio...
Name: tokens, Length: 2000, dtype: object

In [35]:
dictionary = Dictionary(df['tokens'])

In [38]:
dictionary.compactify()   # reassign integer ids to every word
# filter out words that appear in less than 2 documents, thos appearing in more than 97% of corpus, and keep all other words
dictionary.filter_extremes(no_below=2, no_above=0.97, keep_n=None)
dictionary.compactify()   # reassign integer ids to every word, erasing gaps

In [39]:
corpus = [dictionary.doc2bow(tokens) for tokens in df['tokens']]

### Once we have the corpus
1. HDP: We could infer the number of topics in the corpus with an HDP model, but is not always easy to interpret
2. LSI: this model tells us the optimal number of topics by looking at a coherence measurement.
3. Once we have a good estimate for number of topics we can do LDA model
https://medium.com/@armandj.olivares/topic-modeling-on-spanish-text-f7a5e998fb90